In [21]:
import requests
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import geopy.geocoders as geocoders
import pandas as pd
import geocoder
import random
import time
from random import randint

def get_google_maps_url(address, proxy):
  """
  Gets the URL of a location in Google Maps.

  Args:
    address: The address of the location.

  Returns:
    The URL of the location in Google Maps.
  """

  # URL-encode the address.
  address = address.replace(" ", "%20")

  # Create the URL.
  url = "https://www.google.com/maps/search/?api=1&query=" + address

  # Make the request.
  response = requests.get(url)

  # Check the response status code.
  if response.status_code == 200:
    return response.url
  else:
    raise Exception("Error getting Google Maps URL.")



def geocoder_igac(address):
    #address = "CL 65D BIS SUR  77M  68, BOGOTÁ, D.C."
    #address = "asadadadadadadada"
    #url = get_google_maps_url(address, proxy = proxy)

  #  ##url = "https://www.google.com/maps/search/?api=1&query=Calle%2075%20a%2072"
    #req = Request(url,headers={'User-Agent': 'Mozilla/5.0'})
    #webpage = urlopen(req).read()
    ##print(webpage)

  #  #string_data = webpage.decode('utf-8')

  #  ## Extract the latitude and longitude using regular expressions
    #pattern = r"center=([\d.-]+)%2C([-\d.]+)"
    #match = re.search(pattern, string_data)

  #  #if match:
    #    latitude = float(match.group(1))
    #    longitude = float(match.group(2))
    #    #print("Latitude:", latitude)
    #    #print("Longitude:", longitude)
    #else:
    #    latitude = np.nan
    #    longitude = np.nan
    #    #print("Latitude:", latitude)
    #    #print("Longitude:", longitude)

    geocoder_arcgis = geocoder.arcgis(address)
    geojson_geocoder = geocoder_arcgis.geojson

    if geojson_geocoder["features"] == []:
        Confidence_arcgis = np.nan
        Longitude_arcgis = np.nan
        Latitude_arcgis = np.nan
    else:
        Confidence_arcgis = geojson_geocoder["features"][0]["properties"]["confidence"]
        Longitude_arcgis = geojson_geocoder["features"][0]["geometry"]["coordinates"][0]
        Latitude_arcgis = geojson_geocoder["features"][0]["geometry"]["coordinates"][1]


    # try:
    #   geolocator = geocoders.Nominatim(user_agent="my_app")
    #   location = geolocator.geocode(address)
    #       #"CALLE 74A 114 30, BOGOTÁ"
    #   if location == None:
    #       latitude_nominatim = np.nan
    #       longitude_nominatim = np.nan
    #   else:
    #       latitude_nominatim = location.latitude
    #       longitude_nominatim = location.longitude
    # except:
    #   latitude_nominatim = np.nan
    #   longitude_nominatim = np.nan
    # Initialize the geocoder


    return [address, Latitude_arcgis, Longitude_arcgis, Confidence_arcgis]

#geocoder_igac("Calle 75a N 72b - 71, Bogotá")

In [22]:
ofertas = pd.read_excel("24052023_Consolidado_Avaluos.xlsx", "Hoja1")

pattern = r'[^a-zA-Z0-9]'
# Assuming 'df' is your DataFrame and 'Direccion' is the column name



ofertas['Direccion1'] = ofertas['Direccion'].astype(str).str.replace('AP\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('APT\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('AP\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('APT\d+', ' ', regex = True)

ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('CS\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('CS\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('CASA\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('CASA\d+', ' ', regex = True)

ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('TO\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('TO\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('TORRE\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('TORRE\d+', ' ', regex = True)

ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('BL\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('BL\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('BLOQUE\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('BLOQUE\d+', ' ', regex = True)

ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.extract(r'(\D*\d+\D*\d+\D*\d+)')
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace(pattern, ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace(" NO ", ' ')
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace(' N ', ' ')


# Function to count consecutive letters and numbers without spaces
def count_consecutive_alphanumeric(text):
    consecutive_alphanumeric = re.findall(r'[a-zA-Z0-9]+', text)
    max_length = max(len(word) for word in consecutive_alphanumeric)
    return max_length

# Function to insert spaces into the code
def insert_spaces(code):
    pattern = r'(\d+|\D+)'
    spaced_code = re.sub(pattern, r'\1 ', code).strip()
    return spaced_code


ofertas['Direccion2'] = ofertas['Direccion1'].astype(str) + ", " +ofertas['Municipio_nombre'].astype(str) + ", " + ofertas["Departamento_nombre"].astype(str)
ofertas['Direccion2'] = ofertas['Direccion2'].astype(str).str.replace(', BOGOTÁ, D.C., BOGOTÁ, D.C.', ', BOGOTÁ D.C.')
ofertas['Direccion2'] = ofertas['Direccion2'].astype(str).apply(lambda x: re.sub(r'\s{2,}', ' ', x.strip()))
#ofertas['Direccion1'] = ofertas['Direccion1'].str.replace(', BOGOTÁ D.C., BOGOTÁ D.C.', ', BOGOTÁ D.C.')
list_directions = ofertas['Direccion2'].unique().tolist()
sorted_list = sorted(map(str, list_directions))
#sorted_list = [re.sub(r'\s{2,}', ' ', x.strip()) for x in sorted_list]
#sorted_list = sorted(map(str, sorted_list))


In [23]:
len(sorted_list)

176381

In [14]:
ofertas[["Direccion1", "Direccion2", "Direccion"]].sort_values("Direccion2").head(50)

,Direccion1,Direccion2,Direccion
37137,\t PRIMAVERA 6-39,"\t PRIMAVERA 6-39, BOGOTÁ, D.C., BOGOTÁ, D.C.",\t PRIMAVERA 6-39
448184,\tCL 57 C SUR 81 D 03,"\tCL 57 C SUR 81 D 03, BOGOTÁ, D.C., BOGOTÁ, D.C.",\tCL 57 C SUR 81 D 03 AP 802 IN 4
107000,15 EDF ELCONQUISTADOR C 1,"15 EDF ELCONQUISTADOR C 1, CARTAGENA DE I...",AP8 15 EDF ELCONQUISTADOR C1A 3 189 ...
195467,,", ARMENIA, QUINDÍO",APT 10 APT 204 ...
203452,A CIPRES DIAG 25,"A CIPRES DIAG 25, DOSQUEBRADAS, RISARALDA",AP1103 AP6A CIPRES DIAG 25 24T 71 ...
107956,AV PRIMERA 8 50,"AV PRIMERA 8 50, CARTAGENA DE INDIAS, BOLÍVAR",AP604 AV PRIMERA 8 50 PALMETTO ...
107643,BL 2,"BL 2, CARTAGENA DE INDIAS, BOLÍVAR",AP 1604 BL 2 TORRES DE LA PLAZUELA ...
125869,BL 3 CL 12,"BL 3 CL 12, SANTA MARTA, MAGDALENA",AP 604 BL 3 CL 12 18 122 CONDO PAL ...
114981,BL W ET5,"BL W ET5, TURBACO, BOLÍVAR",AP412 BL W ET5 BRISAS DE GALICIA ...
199578,BQ36,"BQ36, PEREIRA, RISARALDA",AP 3642 BQ36 ET J MIRADOR DE LLANO ...


In [19]:
my_min = 0
my_max = 20000#25000

indexes = list(range(my_min, my_max))
my_list = sorted_list[my_min:my_max]

# Define the chunk size
chunk_size = 5000
df = []
# Iterate over the list and apply the function
for i, element in enumerate(my_list):
    result = geocoder_igac(element)
    result = result + [indexes[i]]
    
    df.append(result)
     # Save the results to a CSV file every chunk_size elements
    if (i % chunk_size == 0 or i == (len(my_list)-1)) & (i != 0):
        # Create a DataFrame with the results
        #df = pd.DataFrame(result, columns=['Result'])       
        
        # Generate the CSV file name
        file_name = f'output/Results_finales_{i}.csv'

        # Save the DataFrame to a CSV file
        pd.DataFrame(df).to_csv(file_name, index=False)

Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)


In [23]:

# Generate the CSV file name
file_name = f'output/Results_{i}.csv'

# Save the DataFrame to a CSV file
pd.DataFrame(df).to_csv(file_name, index=False)

In [22]:
pd.DataFrame(df).shape

(25000, 6)